In [1]:
import os 
import sys

import FINE as fn
import FINE.IOManagement.xarrayIO as xrIO 

%load_ext autoreload
%autoreload 2

# How to save an energy system model instance and set it back up? 
# --> xarray and NetCDF files to the rescue! 

The data contained within an Energy System Model (ESM) instance is vast and complex. Saving it directly is not possible. It can, however, be saved as a NetCDF file which supports complex data structures. 

#### What exactly is NetCDF? 
NetCDF (Network Common Data Format) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. It is also a community standard for sharing scientific data. 

#### Python modules that support working with NetCDF files:
1. netcdf4-python: Official Python interface to netCDF files
2. PyNIO: To access different file formats such as netCDF, HDF, and GRIB
3. xarray: Based on NumPy and pandas

Note: xarray module is used here. 

For our use case, the following functionalities are provided: 
* Conversion of ESM instance to xarray dataset. Additionally, possible to save this dataset as NetCDF file in a desired folder, with a desired file name. 
* Conversion of xarray dataset/saved NetCDF file back to ESM instance.

#### Structure of the xarray dataset: 

<img src="xarray_fine.png" style="width: 1000px;"/>


## Conversion of ESM instance to xarray dataset and saving it as a NetCDF file

#### STEP 1. Set up your  ESM instance 

In [2]:
sys.path.append(os.path.join(os.getcwd(), '..', 'Multi-regional_Energy_System_Workflow'))
from getData import getData

data = getData()

# 1. Create an energy system model instance
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                     'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}

esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

CO2_reductionTarget = 1


# 2. Add commodity sources to the energy system model
### Wind onshore

esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

### PV

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))


# 3. Add conversion components to the energy system model

### New combined cycly gas turbines for hydrogen
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True,
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))


# 4. Add commodity storages to the energy system model

### Lithium ion batteries
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

### Hydrogen filled salt caverns
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))


# 5. Add commodity transmission components to the energy system model

### AC cables
esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

### DC cables
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))


### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

# 6. Add commodity sinks to the energy system model

### Electricity demand
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))


The distances of a component are set to a normalized value of 1.


 c:\users\s.patil\documents\code\fine\FINE\utils.py:671: UserWarning:A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.


#### STEP 2. Conversion to xarray dataset and saving as NetCDF file

In [3]:
# save = True saves the file. file_name can be a full path too. Note the file extension '.nc4'
xr_dataset = xrIO.convertEsmInstanceToXarrayDataset(esM, save = True, file_name = 'my_esm_instance.nc4')

 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [4]:
xr_dataset

<xarray.Dataset>
Dimensions:                                    (component: 11, space: 8, space_2: 8, time: 8760)
Coordinates:
  * component                                  (component) object 'Conversion...
  * time                                       (time) int64 0 1 2 ... 8758 8759
  * space                                      (space) object 'cluster_0' ......
  * space_2                                    (space_2) object 'cluster_0' ....
Data variables:
    ts_operationRateMax                        (component, time, space) float64 ...
    ts_operationRateFix                        (component, time, space) float64 ...
    2d_locationalEligibility                   (component, space, space_2) float64 ...
    2d_capacityMax                             (component, space, space_2) float64 ...
    2d_investPerCapacity                       (component, space, space_2) float64 ...
    2d_investIfBuilt                           (component, space, space_2) float64 ...
    2d_opexPerOperation                        (component, space, space_2) float64 ...
    2d_opexPerCapacity                         (component, space, space_2) float64 ...
    2d_opexIfBuilt                             (component, space, space_2) float64 ...
    2d_QPcostScale                             (component, space, space_2) float64 ...
    2d_interestRate                            (component, space, space_2) float64 ...
    2d_economicLifetime                        (component, space, space_2) float64 ...
    2d_technicalLifetime                       (component, space, space_2) float64 ...
    2d_reactances                              (component, space, space_2) float64 ...
    2d_losses                                  (component, space, space_2) float64 ...
    2d_distances                               (component, space, space_2) float64 ...
    2d_capacityFix                             (component, space, space_2) float64 ...
    1d_locationalEligibility                   (component, space) float64 1.0...
    1d_capacityMax                             (component, space) float64 nan...
    1d_investPerCapacity                       (component, space) float64 0.5...
    1d_investIfBuilt                           (component, space) float64 0.0...
    1d_opexPerOperation                        (component, space) float64 0.0...
    1d_commodityCost                           (component, space) float64 nan...
    1d_commodityRevenue                        (component, space) float64 nan...
    1d_opexPerCapacity                         (component, space) float64 0.0...
    1d_opexIfBuilt                             (component, space) float64 0.0...
    1d_QPcostScale                             (component, space) float64 0.0...
    1d_interestRate                            (component, space) float64 0.0...
    1d_economicLifetime                        (component, space) float64 10....
    1d_technicalLifetime                       (component, space) float64 10....
    1d_opexPerChargeOperation                  (component, space) float64 nan...
    1d_opexPerDischargeOperation               (component, space) float64 nan...
    0d_name                                    (component) object 'Electroylz...
    0d_commodity                               (component) object nan ... 'hy...
    0d_hasCapacityVariable                     (component) bool True ... True
    0d_capacityVariableDomain                  (component) object 'continuous...
    0d_capacityPerPlantUnit                    (component) int64 1 1 1 ... 1 1
    0d_hasIsBuiltBinaryVariable                (component) bool False ... False
    0d_bigM                                    (component) float64 nan ... 300.0
    0d_operationRateFix                        (component) object None ... None
    0d_tsaWeight                               (component) float64 1.0 ... 1.0
    0d_commodityLimitID                        (component) object nan ... nan
    0d_yearlyLimit                             (

In [5]:
## Access data variables : ts_operationRateMax of Wind (onshore)
xr_dataset['ts_operationRateMax'].loc['Source, Wind (onshore)', :, :]

<xarray.DataArray 'ts_operationRateMax' (time: 8760, space: 8)>
array([[0.56967237, 0.98495813, 0.98793005, ..., 0.75801348, 0.98997028,
        0.62397435],
       [0.5820936 , 0.97245044, 0.97625681, ..., 0.75737691, 0.97418685,
        0.66733795],
       [0.60880392, 0.95150468, 0.93641348, ..., 0.75446998, 0.94705804,
        0.69627154],
       ...,
       [0.27079389, 0.58971916, 0.78920677, ..., 0.27398043, 0.59417562,
        0.50701817],
       [0.26664181, 0.54445967, 0.79158684, ..., 0.25692088, 0.52098685,
        0.39664065],
       [0.28495466, 0.49216961, 0.67678796, ..., 0.29189614, 0.49216088,
        0.31303636]])
Coordinates:
    component  <U22 'Source, Wind (onshore)'
  * time       (time) int64 0 1 2 3 4 5 6 ... 8753 8754 8755 8756 8757 8758 8759
  * space      (space) object 'cluster_0' 'cluster_1' ... 'cluster_7'

In [6]:
## Access data variables : 1d_investPerCapacity of Electroylzers
xr_dataset['1d_investPerCapacity'].loc['Conversion, Electroylzers', :]

<xarray.DataArray '1d_investPerCapacity' (space: 8)>
array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
Coordinates:
    component  <U25 'Conversion, Electroylzers'
  * space      (space) object 'cluster_0' 'cluster_1' ... 'cluster_7'

In [7]:
## Access attributes : locations
xr_dataset.attrs.get('locations') 

#Note: some data types such as sets, dicts are converted to lists or other types in order to be able to save successfully.
# These are converted back to their original form when they are read in, so no worries! 

['cluster_0',
 'cluster_1',
 'cluster_2',
 'cluster_3',
 'cluster_4',
 'cluster_5',
 'cluster_6',
 'cluster_7']

## Conversion of xarray dataset to ESM instance

In [8]:
esM_from_xr = xrIO.convertXarrayDatasetToEsmInstance(xr_dataset)

 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:671: UserWarning:A declaration of bigM is not necessary if hasIsBuil

In [9]:
esM_from_xr

In [10]:
esM_from_xr.getComponentAttribute('Wind (onshore)', 'operationRateMax')

space,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
time,,,,,,,,
0,0.569672,0.984958,0.987930,0.851416,0.276660,0.758013,0.989970,0.623974
1,0.582094,0.972450,0.976257,0.853392,0.275148,0.757377,0.974187,0.667338
2,0.608804,0.951505,0.936413,0.848352,0.286690,0.754470,0.947058,0.696272
3,0.637364,0.913621,0.887258,0.833918,0.303492,0.755537,0.901716,0.705269
4,0.659234,0.874988,0.822736,0.817766,0.324501,0.769687,0.834311,0.710336
...,...,...,...,...,...,...,...,...
8755,0.313734,0.673692,0.757970,0.343864,0.155183,0.320427,0.724774,0.426975
8756,0.279877,0.642817,0.740666,0.289150,0.119844,0.294879,0.672460,0.429161
8757,0.270794,0.589719,0.789207,0.272159,0.101342,0.273980,0.594176,0.507018


In [12]:
esM_from_xr.getComponentAttribute('Electroylzers', 'investPerCapacity')

space
cluster_0    0.5
cluster_1    0.5
cluster_2    0.5
cluster_3    0.5
cluster_4    0.5
cluster_5    0.5
cluster_6    0.5
cluster_7    0.5
dtype: float64

In [13]:
esM_from_xr.locations

{'cluster_0',
 'cluster_1',
 'cluster_2',
 'cluster_3',
 'cluster_4',
 'cluster_5',
 'cluster_6',
 'cluster_7'}

## Alternative: Conversion of NetCDF file to ESM instance

In [16]:
# Alternative to giving an xr dataset, you could pass the full path to your NETCDF file 
esM_from_netcdf = xrIO.convertXarrayDatasetToEsmInstance('my_esm_instance.nc4')

 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1094: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:671: UserWarning:A declaration of bigM is not necessary if hasIsBuil

In [17]:
esM_from_netcdf